In [29]:
import argparse
import copy
import os
import os.path as osp
import time
import warnings

import cv2
import mmcv
import torch
from torch.utils.data import Dataset
import torch.distributed as dist
from mmcv import Config, DictAction
from mmcv.runner import get_dist_info, init_dist
from mmcv.utils import get_git_hash
from mmcv.utils import TORCH_VERSION, Registry, build_from_cfg, digit_version

from mmdet import __version__
from mmdet.apis import init_random_seed, set_random_seed, train_detector
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.utils import (collect_env, get_device, get_root_logger,
                         setup_multi_processes, update_data_root)
import platform

In [3]:
if platform.system() != 'Windows':
    # https://github.com/pytorch/pytorch/issues/973
    import resource
    rlimit = resource.getrlimit(resource.RLIMIT_NOFILE)
    base_soft_limit = rlimit[0]
    hard_limit = rlimit[1]
    soft_limit = min(max(4096, base_soft_limit), hard_limit)
    resource.setrlimit(resource.RLIMIT_NOFILE, (soft_limit, hard_limit))

In [18]:
cfg_full = Config.fromfile('/home/rohit/Projects/Face_Det/Models/YuNet/custom_train/libfacedetection.train/configs/yunet_n.py')
update_data_root(cfg_full)

In [19]:
cfg = cfg_full.data.train

In [20]:
print(cfg)

{'type': 'RetinaFaceDataset', 'ann_file': 'data/labelv2/train/labelv2.txt', 'img_prefix': '', 'pipeline': [{'type': 'LoadImageFromFile', 'to_float32': True}, {'type': 'LoadAnnotations', 'with_bbox': True, 'with_keypoints': False}, {'type': 'RandomSquareCrop', 'crop_choice': [0.5, 0.7, 0.9, 1.1, 1.3, 1.5]}, {'type': 'Resize', 'img_scale': (640, 640), 'keep_ratio': False}, {'type': 'RandomFlip', 'flip_ratio': 0.5}, {'type': 'Normalize', 'mean': [0.0, 0.0, 0.0], 'std': [1.0, 1.0, 1.0], 'to_rgb': False}, {'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels', 'gt_bboxes_ignore', 'gt_keypointss']}]}


In [21]:
from mmdet.datasets.dataset_wrappers import (ClassBalancedDataset, ConcatDataset,
                                   MultiImageMixDataset, RepeatDataset)

if isinstance(cfg, (list, tuple)):
    print(1)
elif cfg['type'] == 'ConcatDataset':
    print(2)
elif cfg['type'] == 'RepeatDataset':
    print(3)
elif cfg['type'] == 'ClassBalancedDataset':
    print(4)
elif cfg['type'] == 'MultiImageMixDataset':
    print(5)
elif isinstance(cfg.get('ann_file'), (list, tuple)):
    print(6)
else:
    print(7)

7


In [22]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, cfg):
        self.img_paths = []
        self.labels = []
        f = open(cfg['ann_file'])
        for i in (f):
            if i[0] == '#':
                self.img_paths.append(i.split(' ')[1])
            else:
                temp = []
                while i[0] != 0:
                    temp.append(i.split(' ')[:4])
                self.labels.append(temp)

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [36]:
cfg['ann_file']
f = open(cfg['ann_file'])
for i in (f):
    if  i[0]!= '#':
        print(i.split(' ')[:4])
        break

['730', '377', '946', '674']


In [16]:
f = open("/home/rohit/Projects/Face_Det/Models/YuNet/custom_train/libfacedetection.train/data/labelv2/train/labelv2.txt", "r")
for i in f:
    print(i)
    break

# /home/rohit/Projects/Face_Det/Models/YuNet/custom_train/libfacedetection.train/data/train/500.jpg 1080 1920

